# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
import os
import sys

%load_ext dotenv
%dotenv

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

price_data_dir = os.getenv('PRICE_DATA')
parquet_files = glob('PRICE_DATA'+ '/**.parquet.parquet')




For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
import dask.dataframe as dd

data_path = r'C:\Users\Navneet Choudhary\OneDrive\Documents\DSI\Scaling to Production\scaling_to_production\05_src\data\temp\stock_px.csv'
ddf = dd.read_csv(data_path)

ddf['Close'] = ddf.groupby('ticker')['Close'].shift(1)

ddf['returns'] = (ddf['Close'] / ddf['Close']) - 1

ddf['hi_lo_range'] = ddf['High'] - ddf['Low']
dd_feat = ddf[['ticker', 'Date', 'Close', 'returns', 'hi_lo_range']]




C:\Users\Navneet Choudhary\AppData\Local\Temp\ipykernel_1532\4041689931.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Close'] = ddf.groupby('ticker')['Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
import dask.dataframe as dd
import pandas as pd

# Example Dask DataFrame
data = {'returns': [0.01, 0.02, -0.01, 0.03, 0.04, -0.02, 0.01, 0.05, 0.03, 0.02, 0.01, -0.01]}
ddf = dd.from_pandas(pd.DataFrame(data), npartitions=2)

# Convert to pandas DataFrame
pdf = ddf.compute()

# Add rolling average return calculation
pdf['rolling_avg_return'] = pdf['returns'].rolling(window=10).mean()

print(pdf)



    returns  rolling_avg_return
0      0.01                 NaN
1      0.02                 NaN
2     -0.01                 NaN
3      0.03                 NaN
4      0.04                 NaN
5     -0.02                 NaN
6      0.01                 NaN
7      0.05                 NaN
8      0.03                 NaN
9      0.02               0.018
10     0.01               0.018
11    -0.01               0.015


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No it was not neccessary to covert to pandas, you can perform the rolling average in Dask. 

+ Would it have been better to do it in Dask? Why?

Yes it would be better in Dask because it is more efficent for larger datasets.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.